# Segmenting and Clustering Neighborhoods in Toronto

 > **Applied Data Science Capstone Project:**
 > In this project we have to scrape the content from Wikipedia page, wrangle and clean the data to transform it into pandas dataframe. Once we have a structured format, we will explore and cluster the neighborhoods in the city of Toronto. 

### First part: Transforming data from Wikipedia into pandas dataframe 

In [1]:
# Importing relevent libraries for the project

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import json
from pandas.io.json import json_normalize
!conda install -c anaconda requests --yes
import requests
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge beautifulsoup4
from bs4 import BeautifulSoup
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries loaded!')

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.24  |                0         132 KB  anaconda
    certifi-2020.6.20          |           py38_0         159 KB  anaconda
    openssl-1.1.1g             |       h1de35cc_0         3.4 MB  anaconda
    ------------------------------------------------------------
                                           Total:         3.7 MB

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates                                 pkgs/main --> anaconda
  certifi                                         pkgs/main --> anaconda
  openssl                                         pkgs/main --> anaconda



openssl-1.1.1g       | 3.4 MB    | ##################################### | 1

In [2]:
# Obtaining Wikipedia content using GET request and BeautifulSoup4
wiki_content= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
wiki_soup= BeautifulSoup(wiki_content, 'lxml')

In [3]:
#Defining the dataframe columns
New_columns=['Postal Code', 'Borough', 'Neighborhood']

#Creating a new dataframe
df=pd.DataFrame(columns=New_columns)
df

,Postal Code,Borough,Neighborhood


In [4]:
#Filling the dataframe
wiki_table=wiki_soup.find('table')
wiki_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

In [5]:
# Looping through the table and filling the dataframe one row at a time
for tr in wiki_table.find_all('tr'):
    new_row=[]
    for td in tr.find_all('td'):
        new_row.append(td.text.strip())
    if len(new_row)==3:
            df.loc[len(df)]=new_row
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
# Processing cells that have assigned borough
df_new= df[df.Borough!='Not assigned']
df_new.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
# Creating the resultant dataframe
df_new = df_new.groupby('Postal Code').agg({'Borough':'first','Neighborhood': ', '.join}).reset_index()
df_new

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [8]:
#Printing number of rows in the dataframe
df_new.shape

(103, 3)

### Second part: Getting the latitude and the longitude coordinates of each neighborhood

In [9]:
#Creating the dataframe of the latitudes and the longitute coordinates from the data
df_ll=pd.read_csv('http://cocl.us/Geospatial_data')
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# Combining neighborhood dataset obtained in first part with coordinates dataset
df_new=pd.merge(df_new, df_ll, on='Postal Code')
df_new.head(103)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


### Third part: Exploring and clustering the neighborhoods in Toronto

In [11]:
#Narrowing datasets limited to Toronto
df_DT=df_new[df_new['Borough']=='Downtown Toronto'].reset_index(drop=True)
df_DT.head(20)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752


In [12]:
# Obtaining geographical coordinate of Toronto
address='Toronto'
geolocator=Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The latitude and longitude coordinates for Toronto is:", (latitude,longitude))

The latitude and longitude coordinates for Toronto is: (43.6534817, -79.3839347)


In [13]:
# Creating a map of Toronto with its downtown neighborhood using coordinates values 
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# Adding markers to map_Toronto
for lat, lng, borough, Neighborhood in zip(df_DT['Latitude'], df_DT['Longitude'], df_DT['Borough'], df_DT['Neighborhood']):
    label = '{},{}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='teal',
        fill=True,
        fill_color='#3186cf',
        fill_opacity=0.9,
        parse_html=False).add_to(map_Toronto)  
map_Toronto


In [14]:
#Defining Foursquare credentials and version
CLIENT_ID = 'SXLDBPIUHPUNES5UPH3TRVZQQNAYFPQ0P5WUIZLJJ5GQZXIF' 
CLIENT_SECRET = '0MI4RFTYOGZP5G0FJJV1XKXUM2MHALJ2IXUO0FRYDVRPSBF1' 
ACCESS_TOKEN = 'HUJUQIZL43KIORHPIFNOT4JM2CKYOMPBMU0EYR1DHGTRXVO1' 
VERSION = '20180604'
LIMIT = 30

In [15]:
#Installing Scikit-learn libraries to perform KMeans clustering 
!conda install -c conda-forge scikit-learn --yes
from sklearn.cluster import KMeans

Solving environment: done

# All requested packages already installed.



In [16]:
# Clustering neighborhoods into 5 clusters using K-means
df_DT=df_new[df_new['Borough']=='Downtown Toronto'].reset_index(drop=True)
k=5
df_DT=df_DT.drop(['Postal Code','Borough','Neighborhood'],1)
KMeans_DT=KMeans(n_clusters=k, random_state=0).fit(df_DT)
KMeans_DT.labels_

array([2, 2, 2, 0, 0, 0, 0, 3, 0, 0, 0, 0, 3, 3, 4, 0, 0, 1, 3],
      dtype=int32)

In [17]:
# Merging KMeans label into previous dataset to create resultant dataset for visualization
df_DT=df_new[df_new['Borough']=='Downtown Toronto'].reset_index(drop=True)
df_DT.insert(0,'Cluster number', KMeans_DT.labels_)
df_DT

,Cluster number,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,2,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,2,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
5,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,3,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,0,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,0,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752


In [18]:
#Visualizing the resulting clusters 
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# Setting color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Adding markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_DT['Latitude'], df_DT['Longitude'], df_DT['Neighborhood'], df_DT['Cluster number']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)      
map_clusters